In [1]:
!pip install git+https://github.com/TransformerLensOrg/TransformerLens

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/TransformerLensOrg/TransformerLens to /tmp/pip-req-build-ai6mg5no
  Running command git clone --filter=blob:none --quiet https://github.com/TransformerLensOrg/TransformerLens /tmp/pip-req-build-ai6mg5no
  Resolved https://github.com/TransformerLensOrg/TransformerLens to commit cb5017ad0f30cde0d3ac0b0f863c27fbec964c28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install webdataset
!pip install datasets
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import sys
from pathlib import Path
from io import BytesIO

import torch as t
import numpy as np
import transformer_lens as tl
import webdataset as wds
import datasets
import nltk

In [5]:
opt_small = tl.HookedTransformer.from_pretrained('opt-125m')
gpt2_small = tl.HookedTransformer.from_pretrained('gpt2-small')

/home/terekhov/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model opt-125m into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
text = "This project will look into cross-model similarity in the hidden representations in the residual stream. The main outcomes are"
logits, cache = opt_small.run_with_cache(text)
cache.keys()

dict_keys(['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_resid_pre', 'block

In [7]:
print(opt_small.to_str_tokens(text))
print(gpt2_small.to_str_tokens(text))

['</s>', 'This', ' project', ' will', ' look', ' into', ' cross', '-', 'model', ' similarity', ' in', ' the', ' hidden', ' representations', ' in', ' the', ' residual', ' stream', '.', ' The', ' main', ' outcomes', ' are']
['<|endoftext|>', 'This', ' project', ' will', ' look', ' into', ' cross', '-', 'model', ' similarity', ' in', ' the', ' hidden', ' representations', ' in', ' the', ' residual', ' stream', '.', ' The', ' main', ' outcomes', ' are']


Aha, I'm guessing OPT has a very similar tokenization scheme. I can filter examples where the tokenization of the sentence is the same.

In [8]:
opt_small.W_K.shape

torch.Size([12, 12, 768, 64])

In [10]:
nlayers = opt_small.W_K.shape[0]
mid_layer = nlayers // 2
d_model = opt_small.W_K.shape[2]
cached_value = None

def cache_it(resid, hook):
    global cached_value
    cached_value = resid.detach()
    # print(hook)
    return resid

loss = opt_small.run_with_hooks(text, return_type="loss", fwd_hooks=[(f'blocks.{mid_layer}.hook_resid_post', cache_it)])

In [11]:
cached_value.reshape((-1, d_model)).shape

torch.Size([23, 768])

In [12]:
texts = ["This project will look into cross-model similarity in the hidden representations in the residual stream. The main outcomes are",
         "A powerful AI model that can generate text from a prompt. The model is trained on a large corpus of text data.",
         "This text was generated by Copilot, excluding, ironically, this line"]

writer = wds.TarWriter("../data/test.tar")

Path("test").mkdir(exist_ok=True)
for i, text in enumerate(texts):
    loss = opt_small.run_with_hooks(text, return_type="loss", fwd_hooks=[(f'blocks.{mid_layer}.hook_resid_post', cache_it)])
    writer.write({
        "__key__": f"{i:06d}",
        "text.txt": text,
        "values.npy": cached_value.cpu().numpy()
    })

writer.close()

In [13]:
dataset = wds.WebDataset("../data/test.tar")

/home/terekhov/.local/lib/python3.10/site-packages/webdataset/compat.py:136: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")


In [14]:
for item in iter(dataset):
    print(item["text.txt"])
    # Read from an npy file
    values = np.load(BytesIO(item["values.npy"]))
    print(values.shape)

b'This project will look into cross-model similarity in the hidden representations in the residual stream. The main outcomes are'
(1, 23, 768)
b'A powerful AI model that can generate text from a prompt. The model is trained on a large corpus of text data.'
(1, 25, 768)
b'This text was generated by Copilot, excluding, ironically, this line'
(1, 15, 768)


Ok now we want to prepare a large set of sentences from wikipedia.

In [15]:
wiki = datasets.load_dataset("wikipedia", "20220301.en")

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [16]:
len(wiki["train"])

6458670

In [17]:
wiki["train"][0]

{'id': '12',
 'url': 'https://en.wikipedia.org/wiki/Anarchism',
 'title': 'Anarchism',
 'text': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.\n\nHumans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latte

In [18]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/terekhov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/terekhov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [19]:
nltk.sent_tokenize(wiki["train"][0]["text"])

['Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy.',
 'Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful.',
 'As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.',
 'Humans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires.',
 'With the rise of organised hierarchical bodies, scepticism toward authority also rose.',
 'Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment.',
 "During the latter half of the 19th and the first decades of the 20th century, the anarchis

In [20]:
train_frac = 0.8
test_frac = 1.0 - train_frac

train_mask = np.random.rand(len(wiki["train"])) < train_frac
test_mask = ~train_mask

In [21]:
print(train_mask[:10])

[ True  True  True  True  True  True False  True  True  True]


In [22]:
sum(train_mask), sum(test_mask)

(5166223, 1292447)

In [26]:
train_inds = np.where(train_mask)[0]
test_inds = np.where(test_mask)[0]

train_inds = np.random.permutation(train_inds)
test_inds = np.random.permutation(test_inds)

In [27]:
np.savez("../data/wiki_split.npz", train_inds=train_inds, test_inds=test_inds)

In [29]:
train_inds[:10]

array([ 978864, 4705685, 2230602, 2500042, 3619777, 1211633, 2850756,
       5975422, 1238987, 4861382])

In [ ]:
from tqdm.notebook import tqdm

In [35]:
# Check a few random sentences on whether the tokenization is the same
num_samples = 10000
num_diff = 0
for i in tqdm(train_inds[:num_samples]):
    text = wiki["train"][int(i)]["text"]
    sent = nltk.sent_tokenize(text)
    random_sent = np.random.choice(sent)
    # print(random_sent)
    opt_tokens = opt_small.to_str_tokens(random_sent)
    gpt2_tokens = gpt2_small.to_str_tokens(random_sent)
    opt_tokens = opt_tokens[1:]
    gpt2_tokens = gpt2_tokens[1:]
    if opt_tokens != gpt2_tokens:
        num_diff += 1
print(f"Number of different tokenizations: {num_diff}/{num_samples}")

  0%|          | 0/10000 [00:00<?, ?it/s]

Number of different tokenizations: 21/10000
